# Create CNN Model and Optimize it using Keras Tuner

https://www.youtube.com/watch?v=OzLAdpqm35E

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 5.4 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.8.0


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [ ]:
len(train_images)

60000

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
train_images.shape

(60000, 28, 28, 1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_1_filter',min_value=64,max_value=128,step=16),
                                kernel_size=hp.Choice('conve_1_kernel',values=[3,5]),
                                activation='relu',
                                input_shape=(28,28,1)
                            ),
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                                kernel_size=hp.Choice('conve_2_kernel',values=[3,5]),
                                activation='relu',
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units = hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                                activation = 'relu'
                            ),
                            keras.layers.Dense(10,activation='softmax')

  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [ ]:
tuner_search = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='output',
    project_name="Mnist_Fashion"
)

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 4 Complete [00h 15m 22s]
val_accuracy: 0.906333327293396

Best val_accuracy So Far: 0.9101666808128357
Total elapsed time: 00h 50m 08s

Search: Running Trial #5

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |96                |128               
conve_1_kernel    |5                 |3                 
conv_2_filter     |32                |48                
conve_2_kernel    |5                 |3                 
dense_1_units     |48                |64                
learning_rate     |0.001             |0.001             

Epoch 1/3
1688/1688 [==============================] - 388s 229ms/step - loss: 0.4312 - accuracy: 0.8441 - val_loss: 0.3155 - val_accuracy: 0.8822
Epoch 2/3
 913/1688 [===============>..............] - ETA: 2:53 - loss: 0.2923 - accuracy: 0.8940

In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 128)       3328      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        73792     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 112)               3469424   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 3,547,674
Trainable params: 3,547,674
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 20s 12ms/step - loss: 0.1991 - accuracy: 0.9249 - val_loss: 0.2642 - val_accuracy: 0.9103
Epoch 5/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.1524 - accuracy: 0.9437 - val_loss: 0.2832 - val_accuracy: 0.9093
Epoch 6/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.1125 - accuracy: 0.9579 - val_loss: 0.2761 - val_accuracy: 0.9163
Epoch 7/10
 516/1688 [========>.....................] - ETA: 12s - loss: 0.0766 - accuracy: 0.9722